### Advent of Code 2023

This notebook contains my solutions for the Advent of Code (https://adventofcode.com/2023) programming challenge.

#### Day 1: Trebuchet?!
https://adventofcode.com/2023/day/1

In [1]:
# --- Test Input 1 ---

input1 = """1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet"""

# --- Test Input 2 ---

input2 = """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen"""

In [2]:
# --- Part 1 ---

rows = [list(filter(str.isnumeric, line)) for line in input1.splitlines()]
sum(int(n[0]+n[-1]) for n in rows)

142

In [3]:
# --- Part 2 ---

for i,n in enumerate("one two three four five six seven eight nine".split()):
    input2 = input2.replace(n, n[0]+str(i+1)+n[2:])
rows = (list(filter(str.isnumeric, line)) for line in input2.splitlines())
sum(int(n[0]+n[-1]) for n in rows)

281

#### Day 2: Cube Conundrum
https://adventofcode.com/2023/day/2

In [4]:
# --- Test Input ---

input = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"""

In [5]:
# --- Part 1 ---

bag = {"red": 12 , "green": 13 , "blue": 14}
idSum = 0
for game in input.splitlines():
    ID, _, S = game.partition(':')
    possible = True
    for s in S.split(';'):
        for cs in s.split(','):
            n, c = cs.split()
            if int(n) > bag[c]:
                possible = False
    if possible:
        idSum += int(ID.split()[1])
idSum

8

In [6]:
# --- Part 2 ---

score = 0
for game in input.splitlines():
    bag = {"red": 0 , "green": 0 , "blue": 0}
    ID, _, S = game.partition(':')
    possible = True
    for s in S.split(';'):
        for cs in s.split(','):
            n, c = cs.split()
            if int(n) > bag[c]:
                bag[c] = int(n)
    p = 1
    for v in bag.values(): p *= v
    score += p
score

2286

#### Day 3: Gear Ratios
https://adventofcode.com/2023/day/3

In [4]:
# --- Test Input ---

input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [6]:
# --- Part 1 + 2 ---

import numpy, re

chars = numpy.array([list(l)+['.'] for l in input.split()])
numbers = list(map(int, re.findall('\d+', input)))
numIndexMap = numpy.zeros(chars.shape, 'i') - 1
for i,m in enumerate(re.finditer('\d+', input)):
    numIndexMap.flat[m.start(): m.end()] = i

sPart1 = set()
sPart2 = 0
for x in range(chars.shape[0]):
    for y in range(chars.shape[1]):
        if chars[x,y] != '.' and numIndexMap[x,y] == -1:
            sPart1.update(numIndexMap[x-1:x+2,y-1:y+2].ravel())
        if chars[x,y] == '*':
            ind = set(numIndexMap[x-1:x+2,y-1:y+2].ravel())
            neigb = [i for i in ind if i>=0] # neighbors
            if len(neigb) == 2:
                sPart2 += numbers[neigb[0]] * numbers[neigb[1]]

print('Part 1:', sum([numbers[i] for i in sPart1 if i>=0]))
print('Part 2:', sPart2)

Part 1: 4361
Part 2: 467835


#### Day 4: Scratchcards
https://adventofcode.com/2023/day/4

In [10]:
# --- Test Input ---

input = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

In [11]:
# --- Part 1 + 2 ---

lines = input.splitlines()
s = 0  # Part 1
cardCount = [1] * len(lines)  # Part 2
for i,line in enumerate(lines):
    n1, n2 = line.split(':')[-1].split('|')
    points = len(set(n1.split()).intersection(n2.split()))
    s += int(2**(points-1))  # Part 1
    for j in range(i+1,i+1+points): cardCount[j] += cardCount[i] # Part 2
print('part 1:', s)
print('part 2:', sum(cardCount))

part 1: 13
part 2: 30


#### Day 5: If You Give A Seed A Fertilizer
https://adventofcode.com/2023/day/5

In [12]:
# --- Test Input ---

input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [13]:
blocks = input.split('\n\n')
seeds = list(map(int,blocks[0].split(':')[1].split()))
transforms = [[list(map(int, line.split()))
               for line in b.split(':\n')[-1].split('\n')]
               for b in blocks[1:]]

intervals = set(((i,1) for i in seeds))        # -- part 1 --
intervals = set(zip(seeds[::2], seeds[1::2]))  # -- part 2 --

for transform in transforms:
    shiftedInterv = set()
    for yShifted, y, ny in transform:  # y: transformed interval
        restIntervals = set()
        for i,(x,nx) in enumerate(intervals): # x: original interval
            edges = sorted([(x,'x'), (x+nx,'x'), (y,'y'), (y+ny,'y')])
            order = ''.join(s for _,s in edges)
            parts = [(e, edges[i+1][0]-e) for i,(e,_) in enumerate(edges[:-1])]
            task = { # overlap cases
                # case: what to do with the 3 parts (x:keep, +:transform)
                "xxyy": "x  ", # non-overlapping
                "yyxx": "  x", # non-overlapping
                "xyxy": "x+ ", # x left overlap
                "yxyx": " +x", # x right overlap
                "xyyx": "x+x", # y inside x
                "yxxy": " + ", # x inside y
            }[order]
            for trans, (newX, newN) in zip(task, parts):
                if newN > 0:
                    if trans == 'x': restIntervals.add((newX, newN))
                    if trans == '+': shiftedInterv.add((newX+yShifted-y, newN))
        intervals = restIntervals
    intervals = shiftedInterv.union(restIntervals)
print(min(intervals)[0])

46


#### Day 6: Wait For It
https://adventofcode.com/2023/day/6

In [14]:
# --- Test Input ---

input = """Time:      7  15   30
Distance:  9  40  200"""

In [15]:
# inp = input                  # --- part 1 ---
inp = input.replace(' ','')  # --- part 2 ---

import numpy
times = numpy.array(list(map(int, inp.splitlines()[0].split(':')[1].split())),'d')
distances = numpy.array(list(map(int, inp.splitlines()[1].split(':')[1].split())),'d')
tMin = numpy.ceil(times/2 - numpy.sqrt(times**2/4-distances) + 1E-12)
tMax = numpy.floor(times/2 + numpy.sqrt(times**2/4-distances) - 1E-12)
int(numpy.prod(tMax - tMin + 1))


71503

#### Day 7: Camel Cards
https://adventofcode.com/2023/day/7

In [16]:
# --- Test Input ---

input = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

In [17]:
values, joker = "AKQJT98765432"[::-1], '-'  # --- Part 1 ---
values, joker = "AKQT98765432J"[::-1], 'J'  # --- Part 2 ---

ranks = '11111 1112 122 113 23 14 5'.split()
cardList = input.splitlines()
for i,line in enumerate(cardList):
    card, bid = line.split()
    bestValues = sorted((card.count(c), values.index(c), c)
                        for c in set(card) if c != joker)
    bestCard = card.replace(joker, bestValues[-1][2]
                            if bestValues else values[-1])
    rank = ranks.index(''.join(sorted(str(bestCard.count(c))
                                      for c in set(bestCard))))
    card = ''.join([chr(ord('A')+values.index(v)) for v in card])
    cardList[i] = rank, card, int(bid)
sum([line[2]*(i+1) for i,line in enumerate(sorted(cardList))])


5905

#### Day 8: Haunted Wasteland
https://adventofcode.com/2023/day/8

In [18]:
# --- Test Input ---

input1 = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""

input2 = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)"""

This final solution is assuming 3 properties of the data:
1. The system enters the loop already at the second step.
2. The loop has only one z-node
3. The system enters the loop one step after the z-node

In [19]:
import numpy

# input = input1   # --- part 1 ---
input = input2   # --- part 2 ---

d, _, *net = input.split('\n')
net = {(a:=line.split(' = '))[0]: a[1][1:-1].split(', ') for line in net}
nodes = [k for k in net.keys() if k[-1]=='A'] if input != input1 else ['AAA']
solutions = []
for node in nodes:
    iStep = 0
    while node[2] != 'Z':
        node = net[node]['LR'.index(d[iStep%len(d)])]
        iStep += 1
    solutions.append(iStep)
numpy.lcm.reduce(numpy.array(solutions, 'longlong'))

6

#### Day 9: Mirage Maintenance
https://adventofcode.com/2023/day/9

In [20]:
# --- Test Input ---

input = """0 3 6 9 12 15
1 3 6 10 15 21
10 13 16 21 30 45"""


In [21]:
import numpy

lines = input.split('\n')
inp = numpy.array([list(map(int,i.split())) for i in lines])

inp = inp[:,::-1]  # reverse order --- part 2 ---

s = 0
for line in inp:
    start = []
    a = line
    while not numpy.all(a==0):
        start.append(a[0])
        a = numpy.diff(a)
    a = [*a, 0]
    for i in range(len(start)):
        a = numpy.cumsum([start.pop(), *a])
    s += a[-1]
s

2

#### Day 10: Pipe Maze
https://adventofcode.com/2023/day/10

In [22]:
# --- Test Input ---

input = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""


In [23]:
# --- part 1 ---

import numpy

lines = input.split('\n')
a = numpy.array([list(l) for l in lines])
ny, nx = a.shape
(ys,), (xs,) = numpy.where(a == 'S')

up = '|JLS'
down = '|7FS'
left = '-7JS'
right = '-FLS'

n = numpy.zeros((ny, nx), 'i')
iCount = 0
y, x = ys, xs
n[y,x] = -1
for i in range(a.size):
    dx, dy = 0, 0
    if y > 0 and n[y-1,x]==0 and \
         a[y,x] in up and a[y-1,x] in down: dy = -1
    elif y+1 < ny and n[y+1,x]==0 and \
        a[y,x] in down and a[y+1,x] in up: dy = +1
    elif x > 0 and n[y,x-1]==0 and \
        a[y,x] in left and a[y,x-1] in right: dx = -1
    elif x+1 < nx and n[y,x+1]==0 and \
        a[y,x] in right and a[y,x+1] in left: dx = 1
    else:
        print(int(iCount//2+1)) # end of pipe
        break
    if dx or dy:
        y += dy
        x += dx
        iCount += 1
        n[y,x] = iCount
        if iCount==1: y1, x1 = y, x

80


In [24]:
# --- part 2 ---

sType = ''.join(sorted('d u'[y1-ys+1] + 'l r'[x1-xs+1] + \
                       'd u'[y-ys+1]  + 'l r'[x-xs+1])[-2:])
# close loop (cases to be extended for other data)
a[ys,xs] = {'lu': '7', 'du': '|'}[sType]

nTiles = 0
for y in range(ny):
    crossingH1 = False # 1st half of vertical crossing
    crossingH2 = False # 2nd half of vertical crossing
    for x in range(nx):
        if n[y,x]:
            if a[y,x] in down: crossingH1 = not crossingH1
            if a[y,x] in up: crossingH2 = not crossingH2
        elif crossingH1 and crossingH2: # inside
            nTiles += 1
nTiles

10

#### Day 11: Cosmic Expansion
https://adventofcode.com/2023/day/11

In [25]:
# --- Test Input ---

input = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

In [26]:
# dextra = 2        # --- part 1 ---
dextra = 1000000  # --- part 2 ---

import numpy

lines = input.split('\n')
a = numpy.array([list(l) for l in lines])
ny, nx = a.shape
g = numpy.stack(numpy.where(a == '#')).T # list of galaxy coordinates

y = [] # true coordinate y
x = [] # true coordinate x

for coordinate, lines in (y, a), (x, a.T):
    d = 0
    for line in lines:
        d += dextra if (line == '.').all() else 1
        coordinate.append(d)

sum(abs(x[x2]-x[x1]) + abs(y[y2]-y[y1])
    for y1,x1 in g for y2,x2 in g) // 2

82000210

#### Day 12: Hot Springs
https://adventofcode.com/2023/day/12

In [14]:
input = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""

Items **(i,v)** in counter:
*   **i**: arr is confirmed until arr[:i]
*   **v**: current incomplete length
*   **counter[(i,v)]**: count of solutions with same item

cases for **v**:
*   **v == 0**: waiting for new block to start
*   **v > 0**: waiting for block to end

In [13]:
# nParts = 1  # --- part 1 ---
nParts = 5  # --- part 2 ---

from collections import Counter

csum = 0
for line in input.split('\n'):
    row = '?'.join([line.split()[0]]*nParts)
    arr = list(map(int,line.split()[1].split(',')))*nParts
    counter = Counter([(0,0)])
    for s in row+'.':
        for i,v in list(counter.keys()):
            n = counter.pop((i, v))
            if s in '#?': counter[(i,v+1)] += n
            if s in '.?':
                if v == 0:
                    counter[(i,0)] += n
                elif i < len(arr) and arr[i] == v:
                    counter[(i+1,0)] += n
    csum += counter[(len(arr),0)]
csum

525152

#### Day 13: Point of Incidence
https://adventofcode.com/2023/day/13

In [9]:
# --- Test Input ---

input = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""

In [11]:
part = 2  # --- part 1 and 2 ---

import numpy
scoreSum = 0
for block in input.split('\n\n'):
    score = 0
    b = (numpy.array([list(i) for i in block.split()])=='#').astype('i')
    sides = b, b.T, b[::-1], b.T[::-1]
    for iCut, bSides in enumerate(sides):
        for nCut in range(1,len(bSides)-1,2):
            bCut = bSides[nCut:]
            match = bCut == bCut[::-1]
            if (part == 1 and numpy.all(match)) or \
                    (part == 2 and numpy.sum(~match) == 2):
                score = len(bCut)//2 + (nCut if iCut < 2 else 0)
                break
        if score: break
    scoreSum += score * (100 if iCut%2 == 0 else 1)
scoreSum

400

#### Day 14: Parabolic Reflector Dish
https://adventofcode.com/2023/day/14

In [4]:
# --- Test Input ---

input = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

In [5]:
# --- part 1 ---

def transpose(a):
    return '\n'.join(''.join(i) for i in list(map(list, zip(*a.splitlines()))))

def roll(a):
    a = transpose(a)
    while '.O' in a: a = a.replace('.O','O.')
    return transpose(a)

def score(a):
    countO = [i.count('O') for i in a.split()]
    return sum([(i+1)*n for i, n in enumerate(reversed(countO))])

score(roll(input))

136

In [14]:
# --- part 2 ---

def spin(a): # turn clock wise
    return transpose('\n'.join(a.splitlines()[::-1]))

nCycles = 1000000000

history = []
a = input
for i in range(nCycles):
    if a in history:
        iPeriodStart = history.index(a)
        print(f'periodicity found from step {iPeriodStart} to {i}')
        nRemaining = nCycles - iPeriodStart
        iH = iPeriodStart + nRemaining % (i-iPeriodStart)
        print(score(history[iH]))
        break
    history.append(a)
    for i in range(4): # N,W,S,E
        a = spin(roll(a))

periodicity found from step 3 to 10
64


#### Day 15: Lens Library
https://adventofcode.com/2023/day/15

In [4]:
# --- Test Input ---

input = """rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7"""

In [6]:
# --- part 1 ---

def HASH(s, h=0):
    for l in s:
        h = ((h+ord(l)) * 17) % 256
    return h

sum(map(HASH, input.split(',')))

1320

In [14]:
# --- part 2 ---

boxes = [{} for i in range(256)]
for item in input.split(','):
    if '=' in item:
        label, fLen = item.split('=')
        boxes[HASH(label)][label] = int(fLen)
    if '-' in item:
        label = item[:-1]
        box = boxes[HASH(label)]
        if label in box:
            del box[label]

sum([(HASH(label)+1)*(i+1)*box[label] for box in boxes \
      for i, label in enumerate(box)])

145

#### Day 16: The Floor Will Be Lava
https://adventofcode.com/2023/day/16

In [1]:
input = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""

In [3]:
import numpy
chars = numpy.array([list(l) for l in input.split()])
ny, nx = chars.shape

# (x,y,d), d=0:right, d=1:left, d=2:down, d=3:up
frontStart = [(0, y, 0) for y in range(ny)]
frontStart += [(nx-1, y, 1) for y in range(ny)]
frontStart += [(x, 0, 2) for x in range(nx)]
frontStart += [(x, ny-1, 3) for x in range(nx)]

# frontStart = frontStart[:1] # --- part 1 ---

smax = 0
for f in frontStart:
    front = [f]
    history = numpy.zeros((ny, nx, 4), 'bool')
    while front:
        x, y, d = front.pop(0)
        if not ((0 <= x < nx) and (0 <= y < ny)): continue
        if history[y, x, d]: continue
        history[y, x, d] = 1
        c = chars[y, x]
        if c == '|' and d in (0,1):
            front.append((x, y-1, 3))
            front.append((x, y+1, 2))
        elif c == '-' and d in (2,3):
            front.append((x-1, y, 1))
            front.append((x+1, y, 0))
        else:
            if c == '\\': d = [2,3,0,1][d]
            if c == '/': d = [3,2,1,0][d]
            dx = [1,-1,0,0][d]
            dy = [0,0,1,-1][d]
            front.append((x+dx, y+dy, d))
    nEnergized = numpy.sum(numpy.sum(history, axis=2) > 0)
    smax = max(smax, nEnergized)
print(smax)

51


#### Day 17

#### Day 18

#### Day 19

#### Day 20

#### Day 21

#### Day 22

#### Day 23

#### Day 24